In [51]:
import pandas as pd

In [52]:
#Load file with all the recipes
df = pd.read_excel("../data/raw/Recipes.xlsx")

In [53]:
df.head()

,Nombre,Tipo,Link
0,Vegetable Ramen with Mushrooms and Bok Choy,Veg,https://www.goodhousekeeping.com/food-recipes/...
1,Grilled Steak Tortilla Salad,Meat,https://www.goodhousekeeping.com/food-recipes/...
2,Cauliflower Pizza,Veg,https://www.goodhousekeeping.com/food-recipes/...
3,Honey-Ginger Cedar Plank Salmon,Fish,https://www.goodhousekeeping.com/food-recipes/...
4,Traditional Chicken Curry,Meat,https://www.goodhousekeeping.com/food-recipes/...


In [54]:
#Loading previous file
df_previous_recipes = pd.read_csv("../data/processed/previous_recipes.xlsx")
list_columns = list(df_previous_recipes.columns)
for column in list_columns:
    if column == "Unnamed: 0":
        df_previous_recipes = df_previous_recipes.drop("Unnamed: 0",axis=1)

In [55]:
#Removing previous recipes
df = df[~df.Nombre.isin(df_previous_recipes.Nombre.unique())]

In [56]:
#Selecting random recipes
df_veg = df[df.Tipo == "Veg"].sample(n = 2)
df_meat = df[df.Tipo == "Meat"].sample(n = 1)
df_fish = df[df.Tipo == "Fish"].sample(n = 1)
df_pasta = df[df.Tipo == "Pasta"].sample(n = 1)

In [57]:
#putting together recipes
df_final = pd.concat([pd.concat([df_veg, df_meat]), pd.concat([df_fish, df_pasta])]).reset_index(drop=True)

In [58]:
df_final

,Nombre,Tipo,Link
0,Khao pad Thai,Veg,https://www.food.com/recipe/kao-pad-thai-style...
1,Lentils Dahl,Veg,https://nourishplate.com/lentil-dahl-recipe/
2,Pomegranate-Honey Glazed Chicken and Squash,Meat,https://www.goodhousekeeping.com/food-recipes/...
3,Shrimp Packets with Kale Couscous,Fish,https://www.goodhousekeeping.com/food-recipes/...
4,"Orecchiette with cavolo nero, anchovies & pang...",Pasta,https://www.bbcgoodfood.com/recipes/orecchiett...


In [59]:
#Loading data from recipes
pd.concat([df_previous_recipes,df_final]).reset_index(drop=True).to_csv("../data/processed/previous_recipes.xlsx")

# Sending EMAIL

In [60]:
#Creating subject of mail
from datetime import datetime
year, week_num, day_of_week = datetime.today().isocalendar()
subject = "Menu Week " + str(week_num) +" " + str(year)

In [61]:
df_final

,Nombre,Tipo,Link
0,Khao pad Thai,Veg,https://www.food.com/recipe/kao-pad-thai-style...
1,Lentils Dahl,Veg,https://nourishplate.com/lentil-dahl-recipe/
2,Pomegranate-Honey Glazed Chicken and Squash,Meat,https://www.goodhousekeeping.com/food-recipes/...
3,Shrimp Packets with Kale Couscous,Fish,https://www.goodhousekeeping.com/food-recipes/...
4,"Orecchiette with cavolo nero, anchovies & pang...",Pasta,https://www.bbcgoodfood.com/recipes/orecchiett...


In [62]:
body = ""
for i in range(0,df_final.shape[0]):
    body = body + df_final.loc[i, 'Nombre'] + ": " + df_final.loc[i, 'Link'] + "\n"
print(body)

Khao pad Thai : https://www.food.com/recipe/kao-pad-thai-style-fried-rice-135330
Lentils Dahl : https://nourishplate.com/lentil-dahl-recipe/
Pomegranate-Honey Glazed Chicken and Squash: https://www.goodhousekeeping.com/food-recipes/easy/a46930/pomegranate-honey-glazed-chicken-and-squash-recipe/
Shrimp Packets with Kale Couscous: https://www.goodhousekeeping.com/food-recipes/easy/a21602660/shrimp-packets-with-kale-couscous-recipe/
Orecchiette with cavolo nero, anchovies & pangrattato: https://www.bbcgoodfood.com/recipes/orecchiette-cavolo-nero-anchovies-pangrattato



In [63]:
import smtplib

gmail_user = 'sabaku88@gmail.com'
gmail_password = 'ucalhamhnnyyhmqz'

sent_from = gmail_user
to = ['sabaku88@gmail.com', 'chiara.tronconi.91@gmail.com']
#subject = 'OMG Super Important Message'
#body = str(df_final.iloc[0].Nombre) + ": " + str(df_final.iloc[0].Link)

email_text = """\
From: %s
To: %s
Subject: %s

%s
""" % (sent_from, ", ".join(to), subject, body)

try:
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login(gmail_user, gmail_password)
    server.sendmail(sent_from, to, email_text)
    server.close()

    print('Email sent!')
except:
    print('Something went wrong...')

Email sent!


In [10]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1FosaeraWyZVC6VBddXMYXmlpchahQPmDcHFHoi2KJVI',
                                    dest_path='../data/raw/recipes.xlsx',
                                    unzip=True)

In [16]:
import gdown

url = 'https://docs.google.com/spreadsheets/d/1FosaeraWyZVC6VBddXMYXmlpchahQPmDcHFHoi2KJVI'
output = 'recipes.xlsx'
gdown.download(url, output, quiet=False) 


Downloading...
From: https://docs.google.com/spreadsheets/d/1FosaeraWyZVC6VBddXMYXmlpchahQPmDcHFHoi2KJVI
To: /home/carlos/DataScience/week_menu_generator/notebooks/recipes.xlsx
908kB [00:00, 11.7MB/s]


'recipes.xlsx'

In [17]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2 import service_account
import io

credz = {} #put json credentials her from service account or the like
# More info: https://cloud.google.com/docs/authentication

credentials = service_account.Credentials.from_service_account_info(credz)
drive_service = build('drive', 'v3', credentials=credentials)

file_id = '1FosaeraWyZVC6VBddXMYXmlpchahQPmDcHFHoi2KJVI'
request = drive_service.files().get_media(fileId=file_id)
#fh = io.BytesIO() # this can be used to keep in memory
fh = io.FileIO('file.tar.gz', 'wb') # this can be used to write to disk
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))

ModuleNotFoundError: No module named 'googleapiclient'

In [29]:
# import the required libraries
from __future__ import print_function
import pickle
import os.path
import io
import shutil
import requests
from mimetypes import MimeTypes
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
  
class DriveAPI:
    global SCOPES
      
    # Define the scopes
    SCOPES = ['https://www.googleapis.com/auth/drive']
  
    def __init__(self):
        
        # Variable self.creds will
        # store the user access token.
        # If no valid token found
        # we will create one.
        self.creds = None
  
        # The file token.pickle stores the
        # user's access and refresh tokens. It is
        # created automatically when the authorization
        # flow completes for the first time.
  
        # Check if file token.pickle exists
        if os.path.exists('token.pickle'):
  
            # Read the token from the file and
            # store it in the variable self.creds
            with open('token.pickle', 'rb') as token:
                self.creds = pickle.load(token)
  
        # If no valid credentials are available,
        # request the user to log in.
        if not self.creds or not self.creds.valid:
  
            # If token is expired, it will be refreshed,
            # else, we will request a new one.
            if self.creds and self.creds.expired and self.creds.refresh_token:
                self.creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', SCOPES)
                self.creds = flow.run_local_server(port=0)
  
            # Save the access token in token.pickle
            # file for future usage
            with open('token.pickle', 'wb') as token:
                pickle.dump(self.creds, token)
  
        # Connect to the API service
        self.service = build('drive', 'v3', credentials=self.creds)
  
        # request a list of first N files or
        # folders with name and id from the API.
        results = self.service.files().list(
            pageSize=100, fields="files(id, name)").execute()
        items = results.get('files', [])
  
        # print a list of files
  
        print("Here's a list of files: \n")
        print(*items, sep="\n", end="\n\n")
  
    def FileDownload(self, file_id, file_name):
        request = self.service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
          
        # Initialise a downloader object to download the file
        downloader = MediaIoBaseDownload(fh, request, chunksize=204800)
        done = False
  
        try:
            # Download the data in chunks
            while not done:
                status, done = downloader.next_chunk()
  
            fh.seek(0)
              
            # Write the received data to the file
            with open(file_name, 'wb') as f:
                shutil.copyfileobj(fh, f)
  
            print("File Downloaded")
            # Return True if file Downloaded successfully
            return True
        except:
            
            # Return False if something went wrong
            print("Something went wrong.")
            return False
  
    def FileUpload(self, filepath):
        
        # Extract the file name out of the file path
        name = filepath.split('/')[-1]
          
        # Find the MimeType of the file
        mimetype = MimeTypes().guess_type(name)[0]
          
        # create file metadata
        file_metadata = {'name': name}
  
        try:
            media = MediaFileUpload(filepath, mimetype=mimetype)
              
            # Create a new file in the Drive storage
            file = self.service.files().create(
                body=file_metadata, media_body=media, fields='id').execute()
              
            print("File Uploaded.")
          
        except:
              
            # Raise UploadError if file is not uploaded.
            raise UploadError("Can't Upload File.")
  
if __name__ == "__main__":
    obj = DriveAPI()
    i = int(input("Enter your choice: 1 - Download file, 2- Upload File, 3- Exit.\n"))
      
    if i == 1:
        f_id = input("Enter file id: ")
        f_name = input("Enter file name: ")
        obj.FileDownload(f_id, f_name)
          
    elif i == 2:
        f_path = input("Enter full file path: ")
        obj.FileUpload(f_path)
      
    else:
        exit()

FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'

In [7]:
import pywhatkit

# syntax: phone number with country code, message, hour and minutes
pywhatkit.sendwhatmsg('+46722755944', 'Message 1', 11, 45, True, 2)

In 20 Seconds WhatsApp will open and after True Seconds Message will be Delivered!


ValueError: sleep length must be non-negative